In [0]:
import urllib
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

#Monta bucket para acesso via DataBricks

In [0]:
# Define file type
file_type = 'csv'
# Whether the file has a header
first_row_is_header = 'true'
# Delimiter used in the file
delimiter = ','
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option('header', first_row_is_header)\
.option('sep', delimiter)\
.load('/FileStore/tables/iamaws.csv')

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# AWS S3 bucket name
AWS_S3_BUCKET = 'data-mercado-livre-prices'

# Mount name for the bucket
MOUNT_NAME = '/mnt/process-raw-data'

# Source url
SOURCE_URL = 's3n://{0}:{1}@{2}'.format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)

# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[6]: True

In [0]:
# Check if the AWS S3 bucket was mounted successfully
display(dbutils.fs.ls("/mnt/process-raw-data/"))

path,name,size,modificationTime
dbfs:/mnt/process-raw-data/Raw/,Raw/,0,0
dbfs:/mnt/process-raw-data/Refined/,Refined/,0,0
dbfs:/mnt/process-raw-data/Trusted/,Trusted/,0,0


#transformação dos dados

In [0]:
# Le arquivo csv do bucket
file_location = "dbfs:/mnt/process-raw-data/Raw/DataSmartPhonesML20230211.csv"

df = spark.read.format("csv") \
.option("inferSchema", "true") \
.option("header", "true") \
.option("sep", "|") \
.load(file_location)

In [0]:
df_cad = df.select(col("_c0").alias("ItemId"), 
                   col("0").alias("ItemName"), 
                   col("1").alias("Price"), 
                   col("2").alias("OldPrice"),
                   col("3").alias("installment"),
                   col("4").alias("Reviws"))

A expressão regular r"R\$(\d+[.,]\d+)" procura por um padrão que comece com "R$" seguido por um número (com ou sem decimais), separados por uma vírgula ou ponto. A expressão (\d+[.,]\d+) é capturada como grupo de captura e extraída com o método regexp_extract, com o segundo argumento sendo o índice do grupo de captura (1 neste caso, pois é o primeiro e único grupo de captura).

A expressão regular r"(\d+)x" procura por um padrão de número seguido de "x" e captura esse número como um grupo de captura. A expressão regular r"R\$(\d+[.,]\d+)" procura por um padrão de "R$" seguido por um número (com ou sem decimais), separados por uma vírgula ou ponto. Ambos os padrões são capturados como grupos de captura e extraídos com o método regexp_extract, com o segundo argumento sendo o índice do grupo de captura (1 neste caso, pois é o primeiro e único grupo de captura).

A expressão regular r"sem juros" procura pelo padrão "sem juros" na string. O resultado da expressão é capturado como um grupo de captura e extraído com o método regexp_extract, com o segundo argumento sendo o índice do grupo de captura (0 neste caso, pois é o padrão da string inteira).

In [0]:
#trata coluna de preços e parcelas
df_cad = df_cad.withColumn("OldPrice", regexp_extract(df_cad["OldPrice"], r"R\$(\d+[.,]\d+)", 1))\
        .withColumn("installments", regexp_extract(df_cad["installment"], r"(\d+)x", 1)) \
        .withColumn("installmentsPrice", regexp_extract(df_cad["installment"], r"R\$(\d+[.,]\d+)", 1)) \
        .withColumn("interest", regexp_extract(df_cad["installment"], r"sem juros", 0))

Com o método drop, você pode remover uma ou mais colunas do DataFrame. É possível passar o nome da coluna como string ou uma lista de nomes de colunas para remover múltiplas colunas de uma só vez. Além disso, você também pode especificar o argumento columns para remover as colunas pelo índice (ao invés do nome).

In [0]:
#deleta colunas sem interesse
df_cad = df_cad.drop("installment")

In [0]:
df_cad = df_cad.withColumn("nome", regexp_extract(df_cad["ItemName"], r"(.*) Dual", 1)) \
       .withColumn("modelo", regexp_extract(df_cad["ItemName"], r"(.*) (\d+) GB", 1)) \
       .withColumn("Gigas", regexp_extract(df_cad["ItemName"], r"(\d+) GB", 1)) \
       .withColumn("cor", regexp_extract(df_cad["ItemName"], r"\d+ GB (.*) \d+ GB RAM", 1)) \
       .withColumn("memoria_ram", regexp_extract(df_cad["ItemName"], r"\d+ GB RAM", 0))

In [0]:
df_cad.write.csv("dbfs:/mnt/process-raw-data/Refined/DataSmartPhonesML20230211.csv")

#grava os dados particionados por marca
#Particionar seus dados ajuda a otimizar o desempenho ao ler e processar seus dados, pois permite que você acesse somente as partições relevantes para a sua consulta.
#df.write.partitionBy("BrandName").parquet("dbfs:/mnt/process-raw-data/Refined/DataSmartPhonesML20230211.parquet")

In [0]:
display(df_cad)

ItemId,ItemName,Price,OldPrice,Reviws,installments,installmentsPrice,interest,nome,modelo,Gigas,cor,memoria_ram
0,"Smartphone Motorola Moto G32 128gb 4gb Ram 6,5 - Vermelho",1.099,1.368,null,10,"109,90",sem juros,,,,,
1,Samsung Galaxy A03 Core Dual SIM 32 GB black 2 GB RAM,679.0,1.099,4614,10,"67,90",sem juros,Samsung Galaxy A03 Core,Samsung Galaxy A03 Core Dual SIM 32 GB black,32,black,2 GB RAM
2,"Smartphone Motorola Moto G32 128gb 4gb Ram 6,5 - Rose",1.099,1.368,null,10,"109,90",sem juros,,,,,
3,Moto G82 5G Dual SIM 128 GB branco 6 GB RAM,1.899,2.999,234,10,"189,90",sem juros,Moto G82 5G,Moto G82 5G Dual SIM 128 GB branco,128,branco,6 GB RAM
4,Samsung Galaxy A03s Dual SIM 64 GB preto 4 GB RAM,809.0,"988,09",4247,,,,Samsung Galaxy A03s,Samsung Galaxy A03s Dual SIM 64 GB preto,64,preto,4 GB RAM
5,"Smartphone Galaxy A03 Core 32gb 2gb Tela 6,5 Preto Samsung",679.0,1.099,null,10,"67,90",sem juros,,,,,
6,"Smartphone Motorola Moto G32 128gb 4gb Ram 6,5 - Preto",1.099,1.368,null,10,"109,90",sem juros,,,,,
7,Moto G22 128 GB cosmic black 4 GB RAM,1.095,1.339,399,10,"109,50",sem juros,,Moto G22 128 GB cosmic black,128,cosmic black,4 GB RAM
8,"Smartphone Edge 30 Tela 6,5'' 256gb 8gb Ram Grafite Motorola",2.733,3.999,null,10,"273,32",sem juros,,,,,
9,Soyes Xs13 Mini Smartphone 3g Android 6.0 Dual Sim,256.0,,null,8,"32,03",sem juros,Soyes Xs13 Mini Smartphone 3g Android 6.0,,,,
